# Gegraphical data clustering

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim, ArcGIS, Photon
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
geolocator = Nominatim(user_agent='em_nome_do_pai')
# geolocator = Photon()
# geolocator = ArcGIS(username='cwrneiro', password='Chronos.42progoogle', referer='http://www.example.com')
delayed_geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [1]:
df = pd.read_parquet('stock.parquet')#.sample(n=50)
df.shape

(2413, 6)

In [2]:
df['full_address'] = df.street + ', ' + df.neighborhood
df.full_address

0                 Rua Lamute, São João Clímaco, São Paulo
1             Rua Galeno de Castro, Jurubatuba, São Paulo
2             Alameda Nothmann, Campos Elíseos, São Paulo
3        Rua Luís Correia de Melo, Santo Amaro, São Paulo
4                   Rua Jorge Rizzo, Pinheiros, São Paulo
                              ...                        
2408                 Rua Eleutério, Campo Belo, São Paulo
2409          Rua General Jardim, Vila Buarque, São Paulo
2410    Rua Paulo de Avelar, Vila Dom Pedro Ii, São Paulo
2411    Avenida Leonardo da Vinci, Vila Guarani (z Sul...
2412                Rua São Joaquim, Liberdade, São Paulo
Name: full_address, Length: 2413, dtype: object

In [ ]:
def get_lat_long(address: str):
    location = delayed_geocode(address)
    
    if location is not None:
        return location.latitude, location.longitude

    return None, None

In [3]:
'''
df[['latitude', 'longitude']] = df.full_address.apply(lambda address: pd.Series(get_lat_long(address)))
df
'''

df[['latitude', 'longitude']] = None, None

n_rows = 50

for i in range(len(df.index)//n_rows):
    print(f'Getting coordinates for columns {i*n_rows} to {(i+1)*n_rows}')

    df.iloc[i*n_rows:(i+1)*n_rows, [df.columns.get_loc(c) for c in ['latitude', 'longitude']]] = df.iloc[i*n_rows:(i+1)*n_rows].full_address.apply(lambda address: pd.Series(get_lat_long(address)))

    print(f'Success, with {df.iloc[i*n_rows:(i+1)*n_rows].isna().sum()}% na')


    file_path = f'data/chunk_{i}.parquet'
    
    print(f'Saving to {file_path}')

    df.iloc[i*n_rows:(i+1)*n_rows].to_parquet(file_path)

    print('Saved')

Getting coordinates for columns 0 to 50
Success, with suite_area       0
street           0
neighborhood     0
condominium      0
tax              0
asking_price     0
full_address     0
latitude        23
longitude       23
dtype: int64% na
Saving to data/chunk_0.parquet
Saved
Getting coordinates for columns 50 to 100
Success, with suite_area       0
street           0
neighborhood     0
condominium      0
tax              0
asking_price     0
full_address     0
latitude        24
longitude       24
dtype: int64% na
Saving to data/chunk_1.parquet
Saved
Getting coordinates for columns 100 to 150
Success, with suite_area       0
street           0
neighborhood     0
condominium      0
tax              0
asking_price     0
full_address     0
latitude        21
longitude       21
dtype: int64% na
Saving to data/chunk_2.parquet
Saved
Getting coordinates for columns 150 to 200
Success, with suite_area       0
street           0
neighborhood     0
condominium      0
tax              0
asking